In [32]:
import deepchem as dc
import pandas as pd
import pickle
import numpy as np
import warnings

from utils import *
warnings.filterwarnings('ignore')
file = open('tests/tip3p/model_10/results.pickle','rb')
results = pickle.load(file)
stats = results['stats'][0]
stats

{'phy_rmsd': {'train': 1.536399217378134, 'test': 1.5770023501884833},
 'ml_rmsd': {'train': 0.9321493926026675, 'test': 1.0667797852016094},
 'phy_md': {'train': -0.31263701067615657, 'test': -0.34591249999999996},
 'ml_md': {'train': 0.3593255602100937, 'test': 0.36133541802931574},
 'phy_out_rmsd': {'train': 4.620207526313823, 'test': 5.262996912406466},
 'ml_out_rmsd': {'train': 2.9699125298309386, 'test': 3.4479220352523563},
 'params': {'epochs': 500,
  'dropout': 0.4,
  'batch_normalize': False,
  'batch_size': 1000,
  'feat': 'tip3p',
  'kfold': -1,
  'dense_layer_size': 27,
  'graph_conv_layers': [32, 32],
  'model_dir': 'model_10'}}

In [35]:
error = 1
phys_rmsd_train = []
phys_rmsd_test = []
ml_rmsd_train = []
ml_rmsd_test = []
for i in range(100):
    file = open('tests/tip3p/model_'+str(i)+'/results.pickle','rb')
    results = pickle.load(file)
    stats = results['stats'][0]
    phys_rmsd_test.append(stats['phy_rmsd']['test'])
    phys_rmsd_train.append(stats['phy_rmsd']['train'])
    ml_rmsd_test.append(stats['ml_rmsd']['test'])
    ml_rmsd_train.append(stats['ml_rmsd']['train'])
print('physics model: test',np.round(np.mean(phys_rmsd_test),3),'±',np.round(error*np.std(phys_rmsd_test),3),
          'train',np.round(np.mean(phys_rmsd_train),3),'±',np.round(error*np.std(phys_rmsd_train),3))
print('physics + ml: test',np.round(np.mean(ml_rmsd_test),3),'±',np.round(error*np.std(ml_rmsd_test),3),
          'train',np.round(np.mean(ml_rmsd_train),3),'±',np.round(error*np.std(ml_rmsd_train),3))

physics model: test 1.577 ± 0.0 train 1.536 ± 0.0
physics + ml: test 1.084 ± 0.074 train 0.902 ± 0.065
